# CityBikes

## 1. Send a request to CityBikes for the city of your choice. 

In [1]:
# import the packages we will need for this project
import pandas as pd
import numpy as np
import json
import os
import requests
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# pull data from the citybik.es API and store the json file in a variable 
# there are no parameters to enter, nor is a key required
first_url = 'http://api.citybik.es/v2/networks'
city_bikes_request = requests.get(first_url)
city_bikes_request # confirm the response works

<Response [200]>

In [3]:
# load the request as a JSON file
city_bikes = city_bikes_request.json()

In [4]:
# the json file returned is a list of nested dictionaries of companies responsible for bike rentals
# we will loop through the json to isolate the company that does this for L.A
los_angeles_city_bike_companies = [] # create an empty list for city bike companies based in L.A
for company in city_bikes['networks']: # begin a for loop that loops through each element of the JSON file
    company_city = company['location']['city'] # extract the city that the company is located in and store it in a variable
    if company_city == 'Los Angeles, CA': 
        los_angeles_city_bike_companies.append(company) # append only companies based in L.A to our list for analysis

los_angeles_city_bike_companies

[{'company': ['Los Angeles County Metropolitan Transportation Authority (Metro)',
   'Bicycle Transit Systems',
   'BCycle, LLC'],
  'gbfs_href': 'https://gbfs.bcycle.com/bcycle_lametro/gbfs.json',
  'href': '/v2/networks/metro-bike-share',
  'id': 'metro-bike-share',
  'location': {'city': 'Los Angeles, CA',
   'country': 'US',
   'latitude': 34.04855,
   'longitude': -118.25905},
  'name': 'Metro Bike Share'}]

#### There is one network company in Los Angeles that administers the city bik.es program

## 2. Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

#### According to the documentation page of the City Bik.es API, if we want to pull station data for a specific company, we create a new API request with the network id as the end point

##### The network id for Los Angeles is "metro-bike-share"

In [5]:
second_url = 'http://api.citybik.es/v2/networks/metro-bike-share'
stations_request = requests.get(second_url)
stations_request # confirm the request worked

<Response [200]>

In [6]:
stations = stations_request.json() # load the request as a JSON

In [7]:
# to access the details for each station, we parse the json dictionary returned in stations
stations['network']['stations']

[{'empty_slots': 13,
  'extra': {'address': '699 S Imperial St',
   'last_updated': 1693354115,
   'rental_uris': {'android': 'https://www.bcycle.com/applink?system_id=bcycle_lametro&station_id=bcycle_lametro_3037&platform=android',
    'ios': 'https://www.bcycle.com/applink?system_id=bcycle_lametro&station_id=bcycle_lametro_3037&platform=iOS'},
   'renting': 1,
   'returning': 1,
   'uid': 'bcycle_lametro_3037'},
  'free_bikes': 7,
  'id': 'b6faafb631c1e75fe42dcf0183d5cdd9',
  'latitude': 34.0348,
  'longitude': -118.23128,
  'name': 'Imperial & 7th',
  'timestamp': '2023-08-30T00:08:35.679000Z'},
 {'empty_slots': 7,
  'extra': {'address': '1301 Willow Street',
   'last_updated': 1693354115,
   'rental_uris': {'android': 'https://www.bcycle.com/applink?system_id=bcycle_lametro&station_id=bcycle_lametro_3036&platform=android',
    'ios': 'https://www.bcycle.com/applink?system_id=bcycle_lametro&station_id=bcycle_lametro_3036&platform=iOS'},
   'renting': 1,
   'returning': 1,
   'uid': 

## 3. Put your parsed results into a DataFrame.

### Let's view one entry of the stations json to confirm the column names for the dataframe

In [8]:
stations['network']['stations'][9]

{'empty_slots': 20,
 'extra': {'address': '201 N Los Angeles Street',
  'last_updated': 1693354115,
  'rental_uris': {'android': 'https://www.bcycle.com/applink?system_id=bcycle_lametro&station_id=bcycle_lametro_3016&platform=android',
   'ios': 'https://www.bcycle.com/applink?system_id=bcycle_lametro&station_id=bcycle_lametro_3016&platform=iOS'},
  'renting': 1,
  'returning': 1,
  'uid': 'bcycle_lametro_3016'},
 'free_bikes': 9,
 'id': '87062a6c7eb66fcccf3125407f57b221',
 'latitude': 34.0529,
 'longitude': -118.24156,
 'name': 'Los Angeles & Temple',
 'timestamp': '2023-08-30T00:08:35.798000Z'}

### We will create a list of nested information from each station in the stations json

In [9]:
stations_list = []
for station in stations['network']['stations']:
    stations_information = {
        'station_name' : station['name'],
        'station_id' : station['id'],
        'station_address' : station['extra']['address'],
        'empty_slots' : station['empty_slots'],
        'renting' : station['extra']['renting'],
        'returning' : station['extra']['returning'],
        'uid' : station['extra']['uid'],
        'free_bikes' : station['free_bikes'],
        'latitude' : station['latitude'],
        'longitude' : station['longitude'],
        'timestamp' : station['timestamp']
    }
    stations_list.append(stations_information)
stations_df = pd.DataFrame(stations_list)      

In [10]:
stations_df

,station_name,station_id,station_address,empty_slots,renting,returning,uid,free_bikes,latitude,longitude,timestamp
0,Imperial & 7th,b6faafb631c1e75fe42dcf0183d5cdd9,699 S Imperial St,13,1,1,bcycle_lametro_3037,7,34.03480,-118.23128,2023-08-30T00:08:35.679000Z
1,Willow St & Mateo St,6ae1d85179bc7085c9d8855ac003f6fb,1301 Willow Street,7,1,1,bcycle_lametro_3036,11,34.03919,-118.23253,2023-08-30T00:08:35.684000Z
2,Hope & 6th,8ef537aeddba68ac15b0c47ff03b0a2c,557 S Hope Street,20,1,1,bcycle_lametro_3032,11,34.04989,-118.25588,2023-08-30T00:08:35.690000Z
3,Hope & 1st,c14da060d71e97130920dde4509d7323,111 N Hope St,10,1,1,bcycle_lametro_3024,7,34.05772,-118.24897,2023-08-30T00:08:35.715000Z
4,Grand & Washington,3eaeb9f13ce28cf2e56d21e7dd2b3b5c,1014 S Grand Ave,9,1,1,bcycle_lametro_3025,6,34.03286,-118.26808,2023-08-30T00:08:35.716000Z
...,...,...,...,...,...,...,...,...,...,...,...
219,Sunset & Virgil,a6ace1fa31df6d7d551cb5626c3a73ab,4459 Sunset Blvd,4,1,1,bcycle_lametro_4303,6,34.09801,-118.28707,2023-08-30T00:08:35.877000Z
220,Hoover & Fountain,d463dde2137b159cee2678c6e9525741,1254 N Hoover St,9,1,1,bcycle_lametro_4438,5,34.09493,-118.28456,2023-08-30T00:08:35.921000Z
221,Pershing Square,2c62b4e6116c60d6d04820adb67152da,532 W Olive Street,12,1,1,bcycle_lametro_3063,9,34.04804,-118.25374,2023-08-30T00:08:35.893000Z
222,Venice & Inglewood,976e229ed7e475f2fb1f6885723fe3fe,12006 Venice Blvd,5,1,1,bcycle_lametro_4555,3,34.00587,-118.42916,2023-08-30T00:08:35.983000Z


In [11]:
stations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   station_name     224 non-null    object 
 1   station_id       224 non-null    object 
 2   station_address  224 non-null    object 
 3   empty_slots      224 non-null    int64  
 4   renting          224 non-null    int64  
 5   returning        224 non-null    int64  
 6   uid              224 non-null    object 
 7   free_bikes       224 non-null    int64  
 8   latitude         224 non-null    float64
 9   longitude        224 non-null    float64
 10  timestamp        224 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 19.4+ KB


In [12]:
stations_df.describe()

,empty_slots,renting,returning,free_bikes,latitude,longitude
count,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000
mean,7.888393,0.991071,0.991071,7.133929,34.050734,-118.327315
std,4.160572,0.094279,0.094279,3.505270,0.043373,0.078574
min,0.000000,0.000000,0.000000,0.000000,33.928460,-118.491340
25%,5.000000,1.000000,1.000000,5.000000,34.026603,-118.410585
50%,7.000000,1.000000,1.000000,7.000000,34.047595,-118.291375
75%,10.000000,1.000000,1.000000,9.000000,34.068880,-118.261115
max,26.000000,1.000000,1.000000,21.000000,34.177650,-118.225410


In [18]:
stations_df.to_csv('stations_df.csv', index=False)